In [18]:
import json
import http.client
from os import getenv
from dotenv import load_dotenv
load_dotenv()

API_FOOTBALL_KEY = getenv('API_FOOTBALL_KEY')

def get_football_data(league_id: int, season: int) -> str:
    conn = http.client.HTTPSConnection("api-football-v1.p.rapidapi.com")

    headers = {
        'X-RapidAPI-Key': API_FOOTBALL_KEY,
        'X-RapidAPI-Host': "api-football-v1.p.rapidapi.com"
    }

    print(headers)

    conn.request("GET", f"/v3/standings?season={season}&league={league_id}", headers=headers)

    res = conn.getresponse()
    data = res.read()

    return data.decode("utf-8")
    
def organize_data(data_json):
    organized_data = {
        "league_info": {
            "id": data_json["response"][0]["league"]["id"],
            "name": data_json["response"][0]["league"]["name"],
            "country": data_json["response"][0]["league"]["country"],
            "logo": data_json["response"][0]["league"]["logo"],
            "flag": data_json["response"][0]["league"]["flag"],
            "season": data_json["response"][0]["league"]["season"],
        },
        "standings": []
    }

    for entry in data_json["response"][0]["league"]["standings"][0]:
        team_data = {
            "rank": entry["rank"],
            "team_id": entry["team"]["id"],
            "team_name": entry["team"]["name"],
            "points": entry["points"],
            "goalsDiff": entry["goalsDiff"],
            "form": entry["form"],
        }
        organized_data["standings"].append(team_data)

    return organized_data


In [19]:
# Reading data from file
with open('data.json', 'r') as f:
    data = json.load(f)

org_data_json = organize_data(data)

# Saving org_data_json to a file
with open('org_data.json', 'w') as f:
    json.dump(org_data_json, f)


### Enviando dados para o banco

In [ ]:
from pymongo import MongoClient
load_dotenv()
MONGODB_URI = getenv('MONGODB_URI')
DB_NAME = getenv('DB_NAME')
COLLECTION_NAME = getenv('COLLECTION_NAME')

client = MongoClient(MONGODB_URI)

# Seleciona o banco de dados e a coleção
db = client[DB_NAME]
collection = db[COLLECTION_NAME]

# Inserindo/Atualizando dados no MongoDB
collection.update_one({"league_info.id": 39, "league_info.season": 2023}, {
    "$set": data}, upsert=True)
